# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [3]:

# import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sqlalchemy import create_engine
import sys

%matplotlib inline

In [4]:
sys.path.append('/home/studyammu/Udacity/data/')

In [5]:
# load data from database
engine = create_engine('sqlite:///../data/disaster_message_categories.db')
df = pd.read_sql_table('message_categories', engine)
# df.dropna(how='any', inplace=True)
X = df['message']
y = df.iloc[:,4:]
categories = y.columns

OperationalError: (sqlite3.OperationalError) unable to open database file

In [ ]:
df[~(df.isnull().any(axis=1))|(df.original.isnull())].shape, df[~(df.isnull().any(axis=1))].shape

In [ ]:
df = df[~(df.isnull().any(axis=1))|((df.original.isnull())&~(df.offer.isnull()))]

### 2. Write a tokenization function to process your text data

In [ ]:
import nltk
import re

In [ ]:
from nltk.tokenize import word_tokenize, WhitespaceTokenizer
from nltk.stem import WordNetLemmatizer

In [ ]:
def tokenize(text):
    
    # get tokens from text
    tokens = WhitespaceTokenizer().tokenize(text)
    lemmatizer = WordNetLemmatizer()
    
    # clean tokens
    processed_tokens = []
    for token in tokens:
        token = lemmatizer.lemmatize(token).lower().strip('!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~')
        token = re.sub(r'\[[^.,;:]]*\]', '', token)
        
        # add token to compiled list if not empty
        if token != '':
            processed_tokens.append(token)
        
    return processed_tokens

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.multioutput import MultiOutputClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer, TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier

In [ ]:
pipeline = Pipeline([('vect', CountVectorizer(tokenizer=tokenize)),
                      ('tfidf', TfidfTransformer()),
                      ('clf', MultiOutputClassifier(RandomForestClassifier()))])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [ ]:
import random
from sklearn.model_selection import train_test_split

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [ ]:
pipeline.fit(X_train, y_train)

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [ ]:

from sklearn.metrics import classification_report, accuracy_score

In [ ]:

y_pred = pipeline.predict(X_test)

In [ ]:
print(classification_report(y_test.iloc[:,1:].values, np.array([x[1:] for x in y_pred]), target_names=categories))

### 6. Improve your model
Use grid search to find better parameters. 

In [ ]:
parameters = {'vect__ngram_range':[(1,2),(2,2)],
            'clf__estimator__n_estimators':[60, 120]
             }

cv = GridSearchCV(pipeline, parameters)

In [ ]:
cv.fit(X_train, y_train)

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [ ]:
y_pred = cv.predict(X_test)

In [ ]:
print(classification_report(y_test.iloc[:,1:].values, np.array([x[1:] for x in y_pred]), target_names=categories))

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [ ]:
from sklearn.pipeline import FeatureUnion
from sklearn.preprocessing import FunctionTransformer

In [ ]:
def compute_text_length(data):
    return np.array([len(text) for text in data]).reshape(-1, 1)

In [ ]:
pipeline = Pipeline([
    ('features', FeatureUnion([('text', Pipeline([('vect', CountVectorizer(tokenizer=tokenize)),
                                                 ('tfidf', TfidfTransformer()),
                                                 ])),
                              ('length', Pipeline([('count', FunctionTransformer(compute_text_length, validate=False))]))]
                             )),
    ('clf', MultiOutputClassifier(RandomForestClassifier()))])

In [ ]:
pipeline.fit(X_train, y_train)
y_pred = pipeline.predict(X_test)

In [ ]:
print(classification_report(y_test.iloc[:,1:].values, np.array([x[1:] for x in y_pred]), target_names=categories))

In [ ]:
parameters = {'features__text__vect__ngram_range':[(1,2),(2,2)],
            'clf__estimator__n_estimators':[50, 100]
             }

In [ ]:
cv = GridSearchCV(pipeline, parameters)
cv.fit(X_train, y_train)

In [ ]:
y_pred = cv.predict(X_test)

In [ ]:
print(accuracy_score(y_test.iloc[:,1:].values, np.array([x[1:] for x in y_pred])))

In [ ]:
print(classification_report(y_test.iloc[:,1:].values, np.array([x[1:] for x in y_pred]), target_names=categories))

### 9. Export your model as a pickle file

In [ ]:
import pickle

In [ ]:
pickle.dump(cv, open('model.p', 'wb'))

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.